In [1]:
f = open("had64.txt")

H64 = []
for i in range(64):
    H64.append([1.0 if i=='+' else -1.0 for i in list(f.readline())[:-1]])
    
print(H64)

# print([[H64[i][j]-H64[j][i] for i in range(64)] for j in range(64)])

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0], [1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0

In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from collections import OrderedDict

class NetworkModel(nn.Module):

    def __init__(self, inputDim):

        # Initialize the network layers.

        super(NetworkModel, self).__init__()

        self.lin1 = nn.Linear(inputDim, inputDim, bias=False)

    def forward(self, x):

        # A forward function
        # Linear function without activation

        x = self.lin1(x)
        
#         F-> WalshSpec
#         WalshSpec = F * W_t

        return x

class TrainModel():

    def __init__(self, model, device, learningRate, inputDim, batchSize, numberOfSteps, epochs):

        self.device = device
        self.net = model.to(self.device)
#         self.optimizer = optim.Adam(self.net.parameters(), lr=learningRate, weight_decay=0.00005)
        self.optimizer = optim.SGD(self.net.parameters(), lr=learningRate, momentum=0.9)
        self.inputDim = inputDim
        self.batchSize = batchSize
        self.numberOfSteps = numberOfSteps
        self.epochs = epochs
        self.H = {}
        self.H[4] = torch.tensor([[1.0, 1.0, 1.0, 1.0], [1.0, -1.0, 1.0, -1.0], [1.0, 1.0, -1.0, -1.0], [1.0, -1.0, -1.0, 1.0]])
        self.H[8] = torch.tensor([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
                            [1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0],
                            [1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0],
                            [1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0],
                            [1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0],
                            [1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0],
                            [1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0],
                            [1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0]])
        self.H[64] = torch.tensor(H64)
        if(self.inputDim<=16):
#             For generating data of suficcient LI inputs:
            while(True):
                self.perm = torch.randperm(pow(2, self.inputDim))
#                 print("Permutation of input numbers i.e. truth tables: \n", self.perm)
                self.data = [[0.0]*(self.inputDim-len(bin(num)[2:]))+[float(i) for i in bin(num)[2:]] for num in self.perm]
                if(np.linalg.matrix_rank(self.data[:self.numberOfSteps])<self.inputDim):
                    print("Rank not large enough, generating data again")
                else:
                    print("Rank is ", np.linalg.matrix_rank(self.data[:self.numberOfSteps]))
                    break
# #             For generating one-hot data:
#             self.data = [[0.0]*i + [1.0] + [0.0]*(self.inputDim-i-1) for i in range(self.inputDim)]
            
        else:
            self.data = [[0.0 if random.random()>0.5 else 1.0 for i in range(self.inputDim)] for i in range(self.numberOfSteps)]
            print("Data generated.")
#             while(True):
#                 if(np.linalg.matrix_rank(self.data[:self.numberOfSteps])<self.inputDim):
#                     print("Rank not large enough, generating data again")
#                 else:
#                     print("Rank is ", np.linalg.matrix_rank(self.data[:self.numberOfSteps]))
#                     break
            
    
    def train(self, ):
        
        H_error = []
        for ep in range(self.epochs):
            
            for step in range(self.numberOfSteps):

                self.optimizer.zero_grad()
                input = torch.tensor(self.data[step])
                output = self.net(input)
                loss = F.mse_loss(torch.matmul(input, self.H[self.inputDim]), output)
                loss.backward()
                self.optimizer.step()
            
            weight_dict = OrderedDict(self.net.named_parameters())
            weightFunction = weight_dict['lin1.weight'].T
            H_error.append(F.mse_loss(self.H[self.inputDim], weightFunction).item())
            if (ep==0 or (ep+1)%1==0):
                print("ep= ", ep, "error:", H_error[-1], "\n", weightFunction)
                
            
        
        print("H_error array: \n", H_error)
        weight_dict = OrderedDict(self.net.named_parameters())
        weightFunction = weight_dict['lin1.weight'].T
        print("Final rounded weight function: \n", torch.round(weightFunction))



    def test(self,n):

#         test_samples = torch.tensor([[float(el) for el in row] for row in torch.randint(0, 2, (n, self.inputDim)).to(self.device)])
        test_samples = torch.tensor(self.data[self.numberOfSteps:self.numberOfSteps+n])    
        preds = self.net(test_samples)
        outputs = torch.matmul(test_samples, self.H[self.inputDim])
        print("Test:")
        for i in range(len(preds)):
            print(outputs[i], "\n", torch.round(preds[i]), "\n\n")


def main():

    # Please change the inputs here
    inputDim = 4

    # Additional inputs may not be required to change.
    learningRate = 0.02
    batchSize = 16
    numberOfSteps = 16
    epochs = 200

    # Additional Initializer
    device = torch.device('cpu')
    model =  NetworkModel(inputDim)
    trainer = TrainModel(model, device, learningRate, inputDim, batchSize, numberOfSteps, epochs)
    trainer.train()
#     trainer.test(10)
    
    print(inputDim, learningRate, batchSize, numberOfSteps, epochs)

if __name__ == '__main__':
    main()

Rank is  4
ep=  0 error: 0.4861185848712921 
 tensor([[ 0.9066,  0.2976,  0.0084,  0.0714],
        [ 0.6047, -0.3408,  0.4983, -0.0774],
        [ 0.7227,  0.1405, -0.4217, -0.2838],
        [ 1.1008,  0.0269, -0.2035,  0.3882]], grad_fn=<PermuteBackward0>)
ep=  1 error: 0.20230382680892944 
 tensor([[ 1.2554,  0.6024,  0.4096,  0.3824],
        [ 1.0407, -0.5690,  0.7251, -0.4535],
        [ 1.1331,  0.4968, -0.5733, -0.5608],
        [ 1.3907, -0.3237, -0.4516,  0.5936]], grad_fn=<PermuteBackward0>)
ep=  2 error: 0.06255581229925156 
 tensor([[ 1.0506,  0.7579,  0.6455,  0.6396],
        [ 0.9142, -0.7827,  0.8296, -0.6921],
        [ 0.9862,  0.6979, -0.7551, -0.7454],
        [ 1.1171, -0.6388, -0.6966,  0.7691]], grad_fn=<PermuteBackward0>)
ep=  3 error: 0.018836837261915207 
 tensor([[ 0.9823,  0.8672,  0.7960,  0.8054],
        [ 0.9003, -0.8912,  0.8978, -0.8295],
        [ 0.9488,  0.8354, -0.8684, -0.8551],
        [ 1.0144, -0.8114, -0.8427,  0.8813]], grad_fn=<PermuteBackw

ep=  46 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  47 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  48 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  49 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBack

ep=  88 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  89 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  90 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  91 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBack

ep=  125 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  126 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  127 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  128 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<Permute

ep=  167 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  168 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  169 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  170 error: 2.822186928597148e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<Permute

In [55]:
import numpy as np

print(random.random())
# print(np.linalg.matrix_rank([[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0], [1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0], [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0], [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]))

0.4294725919849127
